In [58]:
import pandas as pd
import numpy as np
import setuptools
import openml
from sklearn.linear_model import LinearRegression 
import lightgbm as lgbm
import lightgbmlss
import optuna
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestRegressor
from sklearn.gaussian_process.kernels import Matern
from engression import engression, engression_bagged
import torch
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import mahalanobis
from scipy.stats import norm
from sklearn.metrics import mean_squared_error
from rtdl_revisiting_models import MLP, ResNet, FTTransformer
from properscoring import crps_gaussian, crps_ensemble
import random
import gpytorch
import tqdm.auto as tqdm
from lightgbmlss.model import *
from lightgbmlss.distributions.Gaussian import *
from drf import drf

In [59]:
# pip install pandas
# pip install setuptools
# pip install openml
# pip install lightgbm
# pip install optuna
# pip install engression


# numpy-1.26.3 pandas-2.1.4 setuptools-69.0.3 openml lightgbm-4.2.0  optuna-3.5.0

In [60]:
#openml.config.apikey = 'FILL_IN_OPENML_API_KEY'  # set the OpenML Api Key
SUITE_ID = 336 # Regression on numerical features
#SUITE_ID = 337 # Classification on numerical features
#SUITE_ID = 335 # Regression on numerical and categorical features
#SUITE_ID = 334 # Classification on numerical and categorical features
benchmark_suite = openml.study.get_suite(SUITE_ID)  # obtain the benchmark suite

task_id=361072
task = openml.tasks.get_task(task_id)  # download the OpenML task
dataset = task.get_dataset()

X, y, categorical_indicator, attribute_names = dataset.get_data(
        dataset_format="dataframe", target=dataset.default_target_attribute)

Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.


In [62]:
print(X.shape)

(8192, 21)


In [63]:
for task_id in benchmark_suite.tasks:
    task = openml.tasks.get_task(task_id)  # download the OpenML task
    dataset = task.get_dataset()

    X, y, categorical_indicator, attribute_names = dataset.get_data(
            dataset_format="dataframe", target=dataset.default_target_attribute)
    
    print('task: ', task_id, ' shape: ', X.shape)

Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until vers

task:  361072  shape:  (8192, 21)
task:  361073  shape:  (15000, 26)


Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until vers

task:  361074  shape:  (16599, 16)
task:  361076  shape:  (6497, 11)


Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until vers

...

task:  361279  shape:  (8885, 42)


Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.


task:  361280  shape:  (4177, 7)


Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.


task:  361281  shape:  (5465575, 8)


In [67]:
len(benchmark_suite.tasks)

19

In [5]:
# New new implementation
N_CLUSTERS=20
# calculate the mean and covariance matrix of the dataset
mean = np.mean(X, axis=0)
cov = np.cov(X.T)
scaler = StandardScaler()

# transform data to compute the clusters
X_scaled = scaler.fit_transform(X)

kmeans = KMeans(n_clusters=N_CLUSTERS, random_state=0, n_init="auto").fit(X_scaled)
distances=[]
mahalanobis_dist=[]
counts=[]
ideal_len=len(kmeans.labels_)/5
for i in np.arange(N_CLUSTERS):
    distances.append(np.abs(np.sum(kmeans.labels_==i)-ideal_len))
    counts.append(np.sum(kmeans.labels_==i))
    mean_k= np.mean(X.loc[kmeans.labels_==i,:], axis=0)
    mahalanobis_dist.append(mahalanobis(mean_k, mean, np.linalg.inv(cov)))

dist_df=pd.DataFrame(data={'mahalanobis_dist': mahalanobis_dist, 'count': counts}, index=np.arange(N_CLUSTERS))
dist_df=dist_df.sort_values('mahalanobis_dist', ascending=False)
dist_df['cumulative_count']=dist_df['count'].cumsum()
dist_df['abs_diff']=np.abs(dist_df['cumulative_count']-ideal_len)

final=(np.where(dist_df['abs_diff']==np.min(dist_df['abs_diff']))[0])[0]
labelss=dist_df.index[0:final+1].to_list()
labels=pd.Series(kmeans.labels_).isin(labelss)
labels.index=X.index
close_index=labels.index[np.where(labels==False)[0]]
far_index=labels.index[np.where(labels==True)[0]]

X_train = X.loc[close_index,:]
X_test = X.loc[far_index,:]
y_train = y.loc[close_index]
y_test = y.loc[far_index]

# calculate the mean and covariance matrix of the dataset
mean_ = np.mean(X_train, axis=0)
cov_ = np.cov(X_train.T)
scaler_ = StandardScaler()

# transform data to compute the clusters
X_train_scaled = scaler_.fit_transform(X_train)

kmeans_ = KMeans(n_clusters=N_CLUSTERS, random_state=0, n_init="auto").fit(X_train_scaled)
distances_=[]
counts_=[]
mahalanobis_dist_=[]
ideal_len_=len(kmeans_.labels_)/5
for i in np.arange(N_CLUSTERS):
    distances_.append(np.abs(np.sum(kmeans_.labels_==i)-ideal_len_))
    counts_.append(np.sum(kmeans_.labels_==i))
    mean_k_= np.mean(X_train.loc[kmeans_.labels_==i,:], axis=0)
    mahalanobis_dist_.append(mahalanobis(mean_k_, mean_, np.linalg.inv(cov_)))

dist_df_=pd.DataFrame(data={'mahalanobis_dist': mahalanobis_dist_, 'count': counts_}, index=np.arange(N_CLUSTERS))
dist_df_=dist_df_.sort_values('mahalanobis_dist', ascending=False)
dist_df_['cumulative_count']=dist_df_['count'].cumsum()
dist_df_['abs_diff']=np.abs(dist_df_['cumulative_count']-ideal_len_)

final_=(np.where(dist_df_['abs_diff']==np.min(dist_df_['abs_diff']))[0])[0]
labelss_=dist_df_.index[0:final_+1].to_list()
labels_=pd.Series(kmeans_.labels_).isin(labelss_)
labels_.index=X_train.index
close_index_=labels_.index[np.where(labels_==False)[0]]
far_index_=labels_.index[np.where(labels_==True)[0]]

X_train_ = X_train.loc[close_index_,:]
X_val = X_train.loc[far_index_,:]
y_train_ = y_train.loc[close_index_]
y_val = y_train.loc[far_index_]

In [6]:
# Convert data to PyTorch tensors
X_train__tensor = torch.tensor(X_train_.values, dtype=torch.float32)
y_train__tensor = torch.tensor(y_train_.values, dtype=torch.float32)
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val.values, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

# Convert to use GPU if available
if torch.cuda.is_available():
    X_train__tensor = X_train__tensor.cuda()
    y_train__tensor = y_train__tensor.cuda()
    X_train_tensor = X_train_tensor.cuda()
    y_train_tensor = y_train_tensor.cuda()
    X_val_tensor = X_val_tensor.cuda()
    y_val_tensor = y_val_tensor.cuda()
    X_test_tensor = X_test_tensor.cuda()
    y_test_tensor = y_test_tensor.cuda()

# Create flattened versions of the data
y_val_np = y_val.values.flatten()
y_test_np = y_test.values.flatten()

#### Gaussian process

In [6]:
# Set the random seed for reproducibility
seed=10
np.random.seed(seed)
torch.manual_seed(seed)

class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, kernel):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = kernel

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# Define the learning params
training_iterations = 1000

# Define the kernels
kernels = [
    gpytorch.kernels.ScaleKernel(gpytorch.kernels.MaternKernel(nu=0.5, ard_num_dims=X_train_.shape[1])),
    gpytorch.kernels.ScaleKernel(gpytorch.kernels.MaternKernel(nu=1.5, ard_num_dims=X_train_.shape[1])),
    gpytorch.kernels.ScaleKernel(gpytorch.kernels.MaternKernel(nu=2.5, ard_num_dims=X_train_.shape[1])),
    gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel(ard_num_dims=X_train_.shape[1])),
]

best_crps = float('inf')
best_kernel = None

def train(model,X_train_tensor,y_train_tensor):
    iterator = tqdm.tqdm(range(training_iterations), desc="Train")

    for _ in iterator:
        # Zero backprop gradients
        optimizer.zero_grad()
        # Get output from model
        output = model(X_train_tensor)
        # Calc loss and backprop derivatives
        loss = -mll(output, y_train_tensor)
        loss.backward()
        iterator.set_postfix(loss=loss.item())
        optimizer.step()
        torch.cuda.empty_cache()

for kernel in kernels:
    # Initialize the Gaussian Process model and likelihood
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model = ExactGPModel(X_train_tensor, y_train_tensor, likelihood, kernel)

    if torch.cuda.is_available():
        model = model.cuda()

    # Use the adam optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # "Loss" for GPs - the marginal log likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    # Train the model
    train(model,X_train__tensor,y_train__tensor)
    
    # Set the model in evaluation mode
    model.eval()
    likelihood.eval()

    # Make predictions on the validation set
    with torch.no_grad(), gpytorch.settings.fast_pred_var():
        y_pred = model(X_val_tensor)

    # Calculate CRPS
    y_pred_np = y_pred.mean.numpy().flatten()
    y_pred_std_np = y_pred.stddev.numpy().flatten()

    # Calculate the CRPS for each prediction
    crps_values = [crps_gaussian(y_val_np[i], mu=y_pred_np[i], sig=y_pred_std_np[i]) for i in range(len(y_val_np))]

    # Calculate the mean CRPS
    mean_crps = np.mean(crps_values)

    # Update the best kernel if the current kernel has a lower CRPS
    if mean_crps < best_crps:
        best_crps = mean_crps
        best_kernel = kernel

Train:   0%|          | 0/1000 [00:00<?, ?it/s]

Train:   0%|          | 0/1000 [00:00<?, ?it/s]

Train:   0%|          | 0/1000 [00:00<?, ?it/s]

Train:   0%|          | 0/1000 [00:00<?, ?it/s]

In [9]:
# Set the random seed for reproducibility
seed=10
np.random.seed(seed)
torch.manual_seed(seed)

class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = best_kernel

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# Define the learning params
training_iterations = 1000

# Initialize the Gaussian Process model and likelihood
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(X_train_tensor, y_train_tensor, likelihood)

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

if torch.cuda.is_available():
    model = model.cuda()
    likelihood = likelihood.cuda()
    mll = mll.cuda()
    kernel = kernel.cuda()

# Train the model
train(model,X_train_tensor,y_train_tensor)

# Set the model in evaluation mode
model.eval()
likelihood.eval()

# Make predictions on the validation set
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    y_pred = model(X_test_tensor)

# Calculate CRPS
y_pred_np = y_pred.mean.numpy().flatten()
y_pred_std_np = y_pred.stddev.numpy().flatten()

# Calculate the CRPS for each prediction
crps_values = [crps_gaussian(y_test_np[i], mu=y_pred_np[i], sig=y_pred_std_np[i]) for i in range(len(y_test_np))]

# Calculate the mean CRPS
CRPS_GP = np.mean(crps_values)

# Update the best kernel if the current kernel has a lower CRPS
print('CRPS_GP: ', CRPS_GP)

Train:   0%|          | 0/1000 [00:00<?, ?it/s]

CRPS_GP:  59.16435169591171


#### Define train function

In [7]:
def train(model,criterion,loss_Adam,optimizer,training_iterations,X_train_tensor,y_train_tensor):
    iterator = tqdm.tqdm(range(training_iterations), desc="Train")

    for _ in iterator:
        # making a pridiction in forward pass
        y_train_hat = model(X_train_tensor).reshape(-1,)
        # calculating the loss between original and predicted data points
        loss = criterion(y_train_hat, torch.Tensor(y_train_tensor))
        # store loss into list
        loss_Adam.append(loss.item())
        # zeroing gradients after each iteration
        optimizer.zero_grad()
        # backward pass for computing the gradients of the loss w.r.t to learnable parameters
        loss.backward()
        # updating the parameters after each iteration
        optimizer.step()
        iterator.set_postfix(loss=loss.item())
        torch.cuda.empty_cache()

#### MLP

In [24]:
N_TRIALS=5

d_out = 1  
d_in=X_train_.shape[1]

def MLP_opt(trial):

    seed=10
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

    n_blocks = trial.suggest_int("n_blocks", 1, 5)
    d_block = trial.suggest_int("d_block", 10, 500)
    dropout = trial.suggest_float("dropout", 0, 1)

    MLP_model = MLP(
    d_in=d_in,
    d_out=d_out,
    n_blocks=n_blocks,
    d_block=d_block,
    dropout=dropout,
    )
    n_epochs=trial.suggest_int('n_epochs', 100, 5000)
    learning_rate=trial.suggest_float('learning_rate', 0.0001, 0.05, log=True)
    weight_decay=trial.suggest_float('weight_decay', 1e-8, 1e-3, log=True)
    optimizer=torch.optim.Adam(MLP_model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    criterion = torch.nn.MSELoss()
    loss_Adam=[]

    if torch.cuda.is_available():
        MLP_model = MLP_model.cuda()
        criterion = criterion.cuda()
    
    train(MLP_model,criterion,loss_Adam,optimizer,n_epochs,X_train__tensor,y_train__tensor)

    # Point prediction
    y_val_hat_MLP = (MLP_model(X_val_tensor).reshape(-1,)).detach().numpy()

    # Estimate standard deviation of the prediction error
    std_dev_error = np.std(y_val - y_val_hat_MLP)

    # Calculate the CRPS for each prediction
    crps_values = [crps_gaussian(y_val_np[i], mu=y_val_hat_MLP[i], sig=std_dev_error) for i in range(len(y_val_hat_MLP))]

    # Calculate the mean CRPS
    mean_crps = np.mean(crps_values)

    return mean_crps

sampler_MLP = optuna.samplers.TPESampler(seed=10)
study_MLP = optuna.create_study(sampler=sampler_MLP, direction='minimize')
study_MLP.optimize(MLP_opt, n_trials=N_TRIALS)

[I 2024-01-26 14:48:28,809] A new study created in memory with name: no-name-ba6b04ff-f31c-476f-8822-5a2fd68c0d23


Train:   0%|          | 0/3769 [00:00<?, ?it/s]

[I 2024-01-26 14:49:03,277] Trial 0 finished with value: 51.87233409758446 and parameters: {'n_blocks': 4, 'd_block': 20, 'dropout': 0.6336482349262754, 'n_epochs': 3769, 'learning_rate': 0.002215416944953109, 'weight_decay': 1.33040303714882e-07}. Best is trial 0 with value: 51.87233409758446.


Train:   0%|          | 0/532 [00:00<?, ?it/s]

[I 2024-01-26 14:49:16,100] Trial 1 finished with value: 855.2130560594854 and parameters: {'n_blocks': 1, 'd_block': 383, 'dropout': 0.16911083656253545, 'n_epochs': 532, 'learning_rate': 0.007075637776590665, 'weight_decay': 0.0005847452881552242}. Best is trial 0 with value: 51.87233409758446.


...

[I 2024-01-26 14:50:06,488] Trial 2 finished with value: 790.857583782651 and parameters: {'n_blocks': 1, 'd_block': 261, 'dropout': 0.8126209616521135, 'n_epochs': 3101, 'learning_rate': 0.008871477434617912, 'weight_decay': 2.879919449586155e-07}. Best is trial 0 with value: 51.87233409758446.


Train:   0%|          | 0/796 [00:00<?, ?it/s]

[I 2024-01-26 14:53:09,235] Trial 3 finished with value: 65.94627040143702 and parameters: {'n_blocks': 5, 'd_block': 360, 'dropout': 0.5425443680112613, 'n_epochs': 796, 'learning_rate': 0.0010177368807699995, 'weight_decay': 2.3478377182859888e-05}. Best is trial 0 with value: 51.87233409758446.


Train:   0%|          | 0/2614 [00:00<?, ?it/s]

[I 2024-01-26 14:56:03,058] Trial 4 finished with value: 52.314464085940614 and parameters: {'n_blocks': 3, 'd_block': 223, 'dropout': 0.6177669784693172, 'n_epochs': 2614, 'learning_rate': 0.005693803629695728, 'weight_decay': 1.0120332166548561e-05}. Best is trial 0 with value: 51.87233409758446.


In [25]:
seed=10
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)

MLP_model = MLP(
    d_in=d_in,
    d_out=d_out,
    n_blocks=study_MLP.best_params['n_blocks'],
    d_block=study_MLP.best_params['d_block'],
    dropout=study_MLP.best_params['dropout'],
    )

if torch.cuda.is_available():
    MLP_model = MLP_model.cuda()
    
n_epochs=study_MLP.best_params['n_epochs']
learning_rate=study_MLP.best_params['learning_rate']
weight_decay=study_MLP.best_params['weight_decay']
optimizer=torch.optim.Adam(MLP_model.parameters(), lr=learning_rate, weight_decay=weight_decay)
criterion = torch.nn.MSELoss()
loss_Adam=[]

train(MLP_model,criterion,loss_Adam,optimizer,n_epochs,X_train_tensor,y_train_tensor)

# Point prediction
y_test_hat_MLP = (MLP_model(X_test_tensor).reshape(-1,)).detach().numpy()

# Estimate standard deviation of the prediction error
std_dev_error = np.std(y_test - y_test_hat_MLP)

# Create a normal distribution for each prediction
pred_distributions = [norm(loc=y_test_hat_MLP[i], scale=std_dev_error) for i in range(len(y_test_hat_MLP))]

# Calculate the CRPS for each prediction
crps_values = [crps_gaussian(y_test_np[i], mu=y_test_hat_MLP[i], sig=std_dev_error) for i in range(len(y_test_hat_MLP))]

# Calculate the mean CRPS
crps_MLP = np.mean(crps_values)

print("CRPS MLP: ", crps_MLP)

Train:   0%|          | 0/3769 [00:00<?, ?it/s]

CRPS MLP:  48.76351590288167


#### ResNet

In [27]:
N_TRIALS=5

d_out = 1  
d_in=X_train_.shape[1]

def ResNet_opt(trial):

    seed=10
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

    n_blocks = trial.suggest_int("n_blocks", 1, 5)
    d_block = trial.suggest_int("d_block", 10, 500)
    dropout1 = trial.suggest_float("dropout1", 0, 1)
    dropout2 = trial.suggest_float("dropout2", 0, 1)
    d_hidden_multiplier=trial.suggest_float("d_hidden_multiplier", 0.5, 3)

    ResNet_model = ResNet(
    d_in=d_in,
    d_out=d_out,
    n_blocks=n_blocks,
    d_block=d_block,
    d_hidden=None,
    d_hidden_multiplier=d_hidden_multiplier,
    dropout1=dropout1,
    dropout2=dropout2,
    )
    if torch.cuda.is_available():
        ResNet_model = ResNet_model.cuda()
    n_epochs=trial.suggest_int('n_epochs', 100, 5000)
    learning_rate=trial.suggest_float('learning_rate', 0.0001, 0.05, log=True)
    weight_decay=trial.suggest_float('weight_decay', 1e-8, 1e-3, log=True)
    optimizer=torch.optim.Adam(ResNet_model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    criterion = torch.nn.MSELoss()
    loss_Adam=[]

    train(ResNet_model,criterion,loss_Adam,optimizer,n_epochs,X_train__tensor,y_train__tensor)

    # Point prediction
    y_val_hat_ResNet = (ResNet_model(X_val_tensor).reshape(-1,)).detach().numpy()

    # Estimate standard deviation of the prediction error
    std_dev_error = np.std(y_val - y_val_hat_ResNet)

    # Calculate the CRPS for each prediction
    crps_values = [crps_gaussian(y_val_np[i], mu=y_val_hat_ResNet[i], sig=std_dev_error) for i in range(len(y_val_hat_ResNet))]

    # Calculate the mean CRPS
    crps_ResNet = np.mean(crps_values)

    return crps_ResNet

sampler_ResNet = optuna.samplers.TPESampler(seed=10)
study_ResNet = optuna.create_study(sampler=sampler_ResNet, direction='minimize')
study_ResNet.optimize(ResNet_opt, n_trials=N_TRIALS)

[I 2024-01-26 14:57:40,330] A new study created in memory with name: no-name-dc43c548-45c6-4e8a-add5-ec068b1bb14f


Train:   0%|          | 0/1201 [00:00<?, ?it/s]

[I 2024-01-26 14:58:03,064] Trial 0 finished with value: 57.87352118978276 and parameters: {'n_blocks': 4, 'd_block': 20, 'dropout1': 0.6336482349262754, 'dropout2': 0.7488038825386119, 'd_hidden_multiplier': 1.7462675307564761, 'n_epochs': 1201, 'learning_rate': 0.000342425210145967, 'weight_decay': 6.348243270946383e-05}. Best is trial 0 with value: 57.87352118978276.


Train:   0%|          | 0/2610 [00:00<?, ?it/s]

[I 2024-01-26 14:58:39,131] Trial 1 finished with value: 13.988067222045578 and parameters: {'n_blocks': 1, 'd_block': 53, 'dropout1': 0.6853598183677972, 'dropout2': 0.9533933461949365, 'd_hidden_multiplier': 0.5098706658197861, 'n_epochs': 2610, 'learning_rate': 0.015604131457893529, 'weight_decay': 1.155128593079557e-05}. Best is trial 1 with value: 13.988067222045578.


...

[I 2024-01-26 15:00:57,785] Trial 2 finished with value: 12.033695503985886 and parameters: {'n_blocks': 4, 'd_block': 153, 'dropout1': 0.9177741225129434, 'dropout2': 0.7145757833976906, 'd_hidden_multiplier': 1.8563609200281532, 'n_epochs': 796, 'learning_rate': 0.0010177368807699995, 'weight_decay': 2.3478377182859888e-05}. Best is trial 2 with value: 12.033695503985886.


Train:   0%|          | 0/3045 [00:00<?, ?it/s]

[I 2024-01-26 15:16:41,351] Trial 3 finished with value: 10.319901510056718 and parameters: {'n_blocks': 3, 'd_block': 223, 'dropout1': 0.6177669784693172, 'dropout2': 0.5131382425543909, 'd_hidden_multiplier': 2.125992954828668, 'n_epochs': 3045, 'learning_rate': 0.014902984681415245, 'weight_decay': 4.057287303826027e-06}. Best is trial 3 with value: 10.319901510056718.


Train:   0%|          | 0/4161 [00:00<?, ?it/s]

[I 2024-01-26 15:27:40,910] Trial 4 finished with value: 12.952644959584417 and parameters: {'n_blocks': 5, 'd_block': 166, 'dropout1': 0.09045934927090737, 'dropout2': 0.30070005663620336, 'd_hidden_multiplier': 0.7849609046588744, 'n_epochs': 4161, 'learning_rate': 0.00013383563361780206, 'weight_decay': 1.3534298216580227e-05}. Best is trial 3 with value: 10.319901510056718.


In [28]:
seed=10
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)

ResNet_model = ResNet(
    d_in=d_in,
    d_out=d_out,
    n_blocks=study_ResNet.best_params['n_blocks'],
    d_block=study_ResNet.best_params['d_block'],
    d_hidden=None,
    d_hidden_multiplier=study_ResNet.best_params['d_hidden_multiplier'],
    dropout1=study_ResNet.best_params['dropout1'],
    dropout2=study_ResNet.best_params['dropout2'],
    )

if torch.cuda.is_available():
    ResNet_model = ResNet_model.cuda()

n_epochs=study_ResNet.best_params['n_epochs']
learning_rate=study_ResNet.best_params['learning_rate']
weight_decay=study_ResNet.best_params['weight_decay']
optimizer=torch.optim.Adam(ResNet_model.parameters(), lr=learning_rate, weight_decay=weight_decay)
criterion = torch.nn.MSELoss()
loss_Adam=[]

train(ResNet_model,criterion,loss_Adam,optimizer,n_epochs,X_train_tensor,y_train_tensor)

# Point prediction
y_test_hat_ResNet = (ResNet_model(X_test_tensor).reshape(-1,)).detach().numpy()

# Estimate standard deviation of the prediction error
std_dev_error = np.std(y_test - y_test_hat_ResNet)

# Calculate the CRPS for each prediction
crps_values = [crps_gaussian(y_test_np[i], mu=y_test_hat_ResNet[i], sig=std_dev_error) for i in range(len(y_test_hat_ResNet))]

# Calculate the mean CRPS
crps_ResNet = np.mean(crps_values)

print("CRPS ResNet: ", crps_ResNet)

Train:   0%|          | 0/3045 [00:00<?, ?it/s]

CRPS ResNet:  15.745058930608618


#### FFTransformer

In [23]:
N_TRIALS=5

def train_trans(model,criterion,loss_Adam,optimizer,training_iterations,X_train_tensor,y_train_tensor):
    iterator = tqdm.tqdm(range(training_iterations), desc="Train")

    for _ in iterator:
        # making a pridiction in forward pass
        y_train_hat = model(X_train_tensor, None).reshape(-1,)
        # calculating the loss between original and predicted data points
        loss = criterion(y_train_hat, torch.Tensor(y_train_tensor))
        # store loss into list
        loss_Adam.append(loss.item())
        # zeroing gradients after each iteration
        optimizer.zero_grad()
        # backward pass for computing the gradients of the loss w.r.t to learnable parameters
        loss.backward()
        # updating the parameters after each iteration
        optimizer.step()
        iterator.set_postfix(loss=loss.item())
        torch.cuda.empty_cache()

d_out = 1  
d_in=X_train_.shape[1]

def FTTrans_opt(trial):

    seed=10
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

    n_blocks = trial.suggest_int("n_blocks", 1, 5)
    d_block_multiplier = trial.suggest_int("d_block_multiplier", 1, 25)
    attention_n_heads = trial.suggest_int("attention_n_heads", 1, 20)
    attention_dropout = trial.suggest_float("attention_dropout", 0, 1)
    ffn_d_hidden_multiplier=trial.suggest_float("ffn_d_hidden_multiplier", 0.5, 3)
    ffn_dropout = trial.suggest_float("ffn_dropout", 0, 1)
    residual_dropout = trial.suggest_float("residual_dropout", 0, 1)

    FTTrans_model = FTTransformer(
    n_cont_features=d_in,
    cat_cardinalities=[],
    d_out=d_out,
    n_blocks=n_blocks,
    d_block=d_block_multiplier*attention_n_heads,
    attention_n_heads=attention_n_heads,
    attention_dropout=attention_dropout,
    ffn_d_hidden=None,
    ffn_d_hidden_multiplier=ffn_d_hidden_multiplier,
    ffn_dropout=ffn_dropout,
    residual_dropout=residual_dropout,
    )

    if torch.cuda.is_available():
        FTTrans_model = FTTrans_model.cuda()

    n_epochs=trial.suggest_int('n_epochs', 100, 5000)
    learning_rate=trial.suggest_float('learning_rate', 0.0001, 0.05, log=True)
    weight_decay=trial.suggest_float('weight_decay', 1e-8, 1e-3, log=True)
    optimizer=torch.optim.Adam(FTTrans_model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    criterion = torch.nn.MSELoss()
    loss_Adam=[]

    train_trans(FTTrans_model,criterion,loss_Adam,optimizer,n_epochs,X_train__tensor,y_train__tensor)

    # Point prediction
    y_val_hat_FTTrans = (FTTrans_model(X_val_tensor, None).reshape(-1,)).detach().numpy()

    # Estimate standard deviation of the prediction error
    std_dev_error = np.std(y_val - y_val_hat_FTTrans)

    # Calculate the CRPS for each prediction
    crps_values = [crps_gaussian(y_val_np[i], mu=y_val_hat_FTTrans[i], sig=std_dev_error) for i in range(len(y_val_hat_FTTrans))]

    # Calculate the mean CRPS
    crps_FTTrans= np.mean(crps_values)

    return crps_FTTrans

sampler_FTTrans = optuna.samplers.TPESampler(seed=10)
study_FTTrans = optuna.create_study(sampler=sampler_FTTrans, direction='minimize')
study_FTTrans.optimize(FTTrans_opt, n_trials=N_TRIALS)

[I 2024-01-26 14:12:45,034] A new study created in memory with name: no-name-89d98ae0-5fc5-4bd5-a718-264130e5fee5


Train:   0%|          | 0/3827 [00:00<?, ?it/s]

[W 2024-01-26 14:48:25,053] Trial 0 failed with parameters: {'n_blocks': 4, 'd_block_multiplier': 1, 'attention_n_heads': 13, 'attention_dropout': 0.7488038825386119, 'ffn_d_hidden_multiplier': 1.7462675307564761, 'ffn_dropout': 0.22479664553084766, 'residual_dropout': 0.19806286475962398, 'n_epochs': 3827, 'learning_rate': 0.0002860388842288948} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\dalma\AppData\Local\Temp\ipykernel_27316\2411606154.py", line 60, in FTTrans_opt
    train_trans(FTTrans_model,criterion,loss_Adam,optimizer,n_epochs,X_train__tensor,y_train__tensor)
  File "C:\Users\dalma\AppData\Local\Temp\ipykernel_27316\2411606154.py", line 8, in train_trans
    y_train_hat = model(X_train_tensor, None).reshape(-1,)
  File "c:\Users\dalma\Desktop\THESIS_ETH_NEW\

KeyboardInterrupt: 

In [ ]:
seed=10
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)

FTTrans_model = FTTransformer(
    n_cont_features=d_in,
    cat_cardinalities=[],
    d_out=d_out,
    n_blocks=study_FTTrans.best_params['n_blocks'],
    d_block=study_FTTrans.best_params['d_block'],
    attention_n_heads=study_FTTrans.best_params['attention_n_heads'],
    attention_dropout=study_FTTrans.best_params['attention_dropout'],
    ffn_d_hidden=None,
    ffn_d_hidden_multiplier=study_FTTrans.best_params['ffn_d_hidden_multiplier'],
    ffn_dropout=study_FTTrans.best_params['ffn_dropout'],
    residual_dropout=study_FTTrans.best_params['residual_dropout'],
    )

if torch.cuda.is_available():
    FTTrans_model = FTTrans_model.cuda()

n_epochs=study_FTTrans.best_params['n_epochs']
learning_rate=study_FTTrans.best_params['learning_rate']
weight_decay=study_FTTrans.best_params['weight_decay']
optimizer=torch.optim.Adam(FTTrans_model.parameters(), lr=learning_rate, weight_decay=weight_decay)
criterion = torch.nn.MSELoss()
loss_Adam=[]

train_trans(FTTrans_model,criterion,loss_Adam,optimizer,n_epochs,X_train_tensor,y_train_tensor)

# Point prediction
y_test_hat_FTTrans = (FTTrans_model(X_test_tensor, None).reshape(-1,)).detach().numpy()

# Estimate standard deviation of the prediction error
std_dev_error = np.std(y_test - y_test_hat_FTTrans)

# Calculate the CRPS for each prediction
crps_values = [crps_gaussian(y_test_np[i], mu=y_test_hat_FTTrans[i], sig=std_dev_error) for i in range(len(y_test_hat_FTTrans))]

# Calculate the mean CRPS
crps_FTTrans= np.mean(crps_values)

print("CRPS FTTrans: ", crps_FTTrans)

#### Boosted trees, random forest, engression, linear regression

In [10]:
N_TRIALS=5

'''def boosted(trial):

    params = {'learning_rate': trial.suggest_float('learning_rate', 0.0001, 0.5, log=True),
              'n_estimators': trial.suggest_int('n_estimators', 100, 500),
              'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
              'max_depth': trial.suggest_int('max_depth', 1, 30),
              'min_child_samples': trial.suggest_int('min_child_samples', 10, 100)}
    
    boosted_tree_model=lgbm.LGBMRegressor(**params)
    boosted_tree_model.fit(X_train_, y_train_)
    y_val_hat_boost=boosted_tree_model.predict(X_val)
    RMSE_boost=np.sqrt(np.mean((y_val-y_val_hat_boost)**2))

    return RMSE_boost

sampler_boost = optuna.samplers.TPESampler(seed=10)
study_boost = optuna.create_study(sampler=sampler_boost, direction='minimize')
study_boost.optimize(boosted, n_trials=N_TRIALS)

boosted_model=lgbm.LGBMRegressor(**study_boost.best_params)

def rf(trial):

    params = {'n_estimators': trial.suggest_int('n_estimators', 100, 500),
              'max_depth': trial.suggest_int('max_depth', 1, 30),
              'max_features': trial.suggest_int('max_features', 1, 30),
              'min_samples_leaf': trial.suggest_int('min_samples_leaf', 10, 100)}
    
    rf_model=RandomForestRegressor(**params)
    rf_model.fit(X_train_, y_train_)
    y_val_hat_rf=rf_model.predict(X_val)
    # Calculate the standard deviation of the residuals
    std_dev = np.std(y_val - y_val_hat_rf)
    # Calculate the CRPS for each prediction
    y_val_np = y_val.values.flatten()
    crps_values = [crps_gaussian(y_val_np[i], mu=y_val_hat_rf[i], sig=std_dev) for i in range(len(y_val_np))]
    CRPS_rf = np.mean(crps_values)

    return CRPS_rf

sampler_rf = optuna.samplers.TPESampler(seed=10)
study_rf = optuna.create_study(sampler=sampler_rf, direction='minimize')
study_rf.optimize(rf, n_trials=N_TRIALS)

rf_model=RandomForestRegressor(**study_rf.best_params)'''

torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
# Create lgb dataset
dtrain_ = lgb.Dataset(torch.tensor(X_train_.values, dtype=torch.float32).clone().detach(), label=y_train_.values)

def boosted(trial):

    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.0001, 0.5, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
        'max_depth': trial.suggest_int('max_depth', 1, 30),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
    }
    opt_params = params.copy()
    n_rounds = opt_params["n_estimators"]
    del opt_params["n_estimators"]
    opt_params['feature_pre_filter']=False

    # Use LightGBMLossGuideRegressor for distributional prediction
    boosted_tree_model = LightGBMLSS(Gaussian(stabilization="None", response_fn="exp", loss_fn="nll"))
    torch.manual_seed(10)
    boosted_tree_model.train(opt_params, dtrain_, num_boost_round=n_rounds)

    # Predict both the mean and standard deviation
    pred_params=boosted_tree_model.predict(X_val, pred_type="parameters")
    y_val_hat_boost=pred_params['loc']
    y_val_hat_std = pred_params['scale']

    # Calculate the CRPS for each prediction
    crps_values = [crps_gaussian(y_val_np[i], mu=y_val_hat_boost[i], sig=y_val_hat_std[i]) for i in range(len(y_val))]

    # Return the mean CRPS as the objective to be minimized
    return np.mean(crps_values)

sampler_boost = optuna.samplers.TPESampler(seed=10)
study_boost = optuna.create_study(sampler=sampler_boost, direction='minimize')
study_boost.optimize(boosted, n_trials=N_TRIALS)

N_SAMPLES=100

def rf(trial):
    params = {'num_trees': trial.suggest_int('num_trees', 100, 500),
          'mtry': trial.suggest_int('mtry', 1, 30),
          'min_node_size': trial.suggest_int('min_node_size', 10, 100)}
    
    drf_model = drf(**params)
    drf_model.fit(X_train_, y_train_)
    
    # Generate a sample from the drf model for each data point
    y_val_hat=drf_model.predict(newdata = X_val, functional = "quantile", quantiles=list(np.random.uniform(0,1,N_SAMPLES)))

    # Calculate the CRPS for each prediction
    crps_values = [crps_ensemble(y_val_np[i], y_val_hat.quantile[i].reshape(-1)) for i in range(len(y_val_np))]

    # Return the mean CRPS as the objective to be minimized
    return np.mean(crps_values)

sampler_drf = optuna.samplers.TPESampler(seed=10)
study_drf = optuna.create_study(sampler=sampler_drf, direction='minimize')
study_drf.optimize(rf, n_trials=N_TRIALS)

def engressor_NN(trial):

    params = {'learning_rate': trial.suggest_float('learning_rate', 0.0001, 0.01, log=True),
              'num_epoches': trial.suggest_int('num_epoches', 100, 1000),
              'num_layer': trial.suggest_int('num_layer', 2, 5),
              'hidden_dim': trial.suggest_int('hidden_dim', 100, 500),}
    params['noise_dim']=params['hidden_dim']
    
    engressor_model=engression(torch.Tensor(np.array(X_train_)), torch.Tensor(np.array(y_train_).reshape(-1,1)), lr=params['learning_rate'], num_epoches=params['num_epoches'],num_layer=params['num_layer'], hidden_dim=params['hidden_dim'], noise_dim=params['noise_dim'], batch_size=1000)
    # Generate a sample from the engression model for each data point
    y_val_hat_engression_samples = [engressor_model.sample(torch.Tensor(np.array([X_val.values[i]])), sample_size=N_SAMPLES) for i in range(len(X_val))]

    # Calculate the CRPS for each prediction
    crps_values = [crps_ensemble(y_val_np[i], np.array(y_val_hat_engression_samples[i]).reshape(-1,)) for i in range(len(y_val_np))]

    return np.mean(crps_values)

sampler_engression = optuna.samplers.TPESampler(seed=10)
study_engression = optuna.create_study(sampler=sampler_engression, direction='minimize')
study_engression.optimize(engressor_NN, n_trials=N_TRIALS)

[I 2024-01-26 16:53:23,764] A new study created in memory with name: no-name-f51e5684-c888-4a7b-b7ad-92a8e391465d


[I 2024-01-26 16:53:27,320] Trial 0 finished with value: 6.937608925970823 and parameters: {'learning_rate': 0.0713003929222653, 'n_estimators': 108, 'reg_lambda': 0.005044685709888605, 'max_depth': 23, 'min_child_samples': 55}. Best is trial 0 with value: 6.937608925970823.
[I 2024-01-26 16:53:29,770] Trial 1 finished with value: 12.527109038809398 and parameters: {'learning_rate': 0.0006784471913345375, 'n_estimators': 179, 'reg_lambda': 0.0699481785242808, 'max_depth': 6, 'min_child_samples': 18}. Best is trial 0 with value: 6.937608925970823.
[W 2024-01-26 16:53:31,431] Trial 2 failed with parameters: {'learning_rate': 0.03428667811840723, 'n_estimators': 482, 'reg_lambda': 1.08526150100961e-08, 'max_depth': 16, 'min_child_samples': 83} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)


KeyboardInterrupt: 

In [12]:
y_train__tensor.reshape(-1,1).shape

torch.Size([5434, 1])

In [17]:
engressor_model=engression(X_train__tensor, y_train__tensor.reshape(-1,1), lr=0.001, num_epoches=5,num_layer=2, hidden_dim=2, noise_dim=2, batch_size=1000)

y_val_hat_engression=engressor_model.predict(X_val_tensor, target="mean")

RMSE_engression=torch.sqrt(torch.mean(torch.square(y_val_tensor.reshape(-1,1) - y_val_hat_engression)))

Running on CPU.

Data is standardized for training only; the printed training losses are on the standardized scale. 
However during evaluation, the predictions, evaluation metrics, and plots will be on the original scale.

Training based on mini-batch gradient descent with a batch size of 1000.
[Epoch 1 (0%), batch 6] energy-loss: 0.8719,  E(|Y-Yhat|): 0.9254,  E(|Yhat-Yhat'|): 0.1071

Training loss on the original (non-standardized) scale:
	Energy-loss: 4.0458,  E(|Y-Yhat|): 4.2826,  E(|Yhat-Yhat'|): 0.4736

Prediction-loss E(|Y-Yhat|) and variance-loss E(|Yhat-Yhat'|) should ideally be equally large
-- consider training for more epochs or adjusting hyperparameters if there is a mismatch 


In [18]:
RMSE_engression

tensor(30.9943)

In [26]:
seed=10
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)

engressor_model=engression(torch.Tensor(np.array(X_train_)), torch.Tensor(np.array(y_train_).reshape(-1,1)), lr=0.001, num_epoches=5,num_layer=2, hidden_dim=2, noise_dim=2, batch_size=1000)
y_val_hat_engression=engressor_model.predict(torch.Tensor(np.array(X_val)), target="mean")
RMSE_engression=np.sqrt((((torch.Tensor(np.array(y_val).reshape(-1,1)))-y_val_hat_engression)**2).mean(axis=0))
print(RMSE_engression)

y_val_hat_engression=engressor_model.predict(X_val_tensor, target="mean")
RMSE_engression=torch.sqrt(torch.mean(torch.square(y_val_tensor.reshape(-1,1) - y_val_hat_engression)))
print(RMSE_engression)

Running on CPU.

Data is standardized for training only; the printed training losses are on the standardized scale. 
However during evaluation, the predictions, evaluation metrics, and plots will be on the original scale.

Training based on mini-batch gradient descent with a batch size of 1000.
[Epoch 1 (0%), batch 6] energy-loss: 0.8607,  E(|Y-Yhat|): 0.9379,  E(|Yhat-Yhat'|): 0.1544

Training loss on the original (non-standardized) scale:
	Energy-loss: 3.4879,  E(|Y-Yhat|): 4.0124,  E(|Yhat-Yhat'|): 1.0489

Prediction-loss E(|Y-Yhat|) and variance-loss E(|Yhat-Yhat'|) should ideally be equally large
-- consider training for more epochs or adjusting hyperparameters if there is a mismatch 
tensor([45.2676])
tensor(45.2691)


In [34]:
pip install git+https://github.com/somepago/saint.git

  Cloning https://github.com/somepago/saint.git to c:\users\dalma\appdata\local\temp\pip-req-build-bqlg_yj2
  Resolved https://github.com/somepago/saint.git to commit e288e84c77a54cfd2ffb55a53678fb7cbbb16630
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/somepago/saint.git 'C:\Users\dalma\AppData\Local\Temp\pip-req-build-bqlg_yj2'
ERROR: git+https://github.com/somepago/saint.git does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [36]:
pip install git+https://github.com/LeoGrin/tabular-benchmark.git

  Cloning https://github.com/LeoGrin/tabular-benchmark.git to c:\users\dalma\appdata\local\temp\pip-req-build-xqta0bl4
  Resolved https://github.com/LeoGrin/tabular-benchmark.git to commit 9d54cf53d9fd3159e367e70a00005f4fcbf2c79d
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/LeoGrin/tabular-benchmark.git 'C:\Users\dalma\AppData\Local\Temp\pip-req-build-xqta0bl4'
  Running command git submodule update --init --recursive -q
ERROR: git+https://github.com/LeoGrin/tabular-benchmark.git does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [42]:
git clone https://github.com/LeoGrin/tabular-benchmark.git

SyntaxError: invalid syntax. Perhaps you forgot a comma? (588987684.py, line 1)

In [49]:
import sys
sys.path.append("C:/Users/dalma/Desktop/THESIS_ETH_NEW/CODE/TABULAR BENCHMARK/tabular-benchmark")

from tabular_benchmark import SaintModel

ModuleNotFoundError: No module named 'tabular_benchmark'

In [50]:
pip install pytorch-widedeep


  Using cached pytorch_widedeep-1.4.0-py3-none-any.whl.metadata (10 kB)
  Using cached gensim-4.3.2-cp310-cp310-win_amd64.whl.metadata (8.5 kB)
  Using cached spacy-3.7.2-cp310-cp310-win_amd64.whl.metadata (26 kB)
  Using cached imutils-0.5.4-py3-none-any.whl
  Using cached torchvision-0.16.2-cp310-cp310-win_amd64.whl.metadata (6.6 kB)
  Using cached einops-0.7.0-py3-none-any.whl.metadata (13 kB)
  Using cached fastparquet-2023.10.1-cp310-cp310-win_amd64.whl.metadata (4.2 kB)
  Using cached smart_open-6.4.0-py3-none-any.whl.metadata (21 kB)
  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl (29 kB)
  Using cached spacy_loggers-1.0.5-py3-none-any.whl.metadata (23 kB)
  Using cached murmurhash-1.0.10-cp310-cp310-win_amd64.whl.metadata (2.0 kB)
  Using cached cymem-2.0.8-cp310-cp310-win_amd64.whl.metadata (8.6 kB)
  Using cached preshed-3.0.9-cp310-cp310-win_amd64.whl.metadata (2.2 kB)
  Using cached thinc-8.2.2-cp310-cp310-win_amd64.whl.metadata (15 kB)
  Using cached wasabi-1.1.2-

In [57]:
from pytorch_widedeep.models import TabTransformer
from pytorch_widedeep.preprocessing import TabPreprocessor
from pytorch_widedeep.training import Trainer

def Saint_opt(trial):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

    n_layers = trial.suggest_int("n_layers", 1, 5)
    n_heads = trial.suggest_int("n_heads", 1, 20)
    embed_dim = trial.suggest_int("embed_dim", 10, 500)
    dropout = trial.suggest_float("dropout", 0, 1)

    # Preprocess the data
    preprocessor = TabPreprocessor(embed_cols=X_train_.columns.tolist())
    X_train__saint = preprocessor.fit_transform(X_train_)
    X_val_saint = preprocessor.transform(X_val)

    embeddings_input = []
    for col in X_train_.columns:
        embeddings_input.append((col, X_train_[col].nunique()))

    # Define the model
    saint_model = TabTransformer(
        column_idx=preprocessor.column_idx,
        input_dim=len(preprocessor.column_idx),
        embed_dim=embed_dim,
        continuous_cols=preprocessor.continuous_cols,
        n_layers=n_layers,
        n_heads=n_heads,
        dropout=dropout,
    )

    # Define the trainer
    trainer = Trainer(saint_model, objective="regression", metrics=[nn.MSELoss()])

    # Train the model
    trainer.fit(
        X_train=X_train__saint,
        target=y_train_,
        n_epochs=trial.suggest_int('n_epochs', 100, 5000),
        #batch_size=trial.suggest_categorical('batch_size', [16, 32, 64, 128]),
        val_split=0.1,
    )

    # Predict on the validation set and calculate the CRPS
    y_val_hat_saint = trainer.predict(X_val_saint)
    std_dev_error = np.std(y_val - y_val_hat_saint)
    crps_saint = [crps_gaussian(y_val_np[i], mu=y_val_hat_saint[i], sig=std_dev_error) for i in range(len(y_val_hat_saint))]
    crps_saint = np.mean(crps_saint)

    return crps_saint

sampler_saint = optuna.samplers.TPESampler(seed=seed)
study_saint = optuna.create_study(sampler=sampler_saint, direction='minimize')
study_saint.optimize(Saint_opt, n_trials=N_TRIALS)

# Preprocess the data
preprocessor = TabPreprocessor(embed_cols=X_train.columns.tolist())
X_train_saint = preprocessor.fit_transform(X_train)
X_test_saint = preprocessor.transform(X_test)

embeddings_input = []
for col in X_train.columns:
    embeddings_input.append((col, X_train[col].nunique()))

# Define the model
saint_model = TabTransformer(
    column_idx=preprocessor.column_idx,
    input_dim=len(preprocessor.column_idx),
    embed_dim=study_saint.best_params['embed_dim'],
    continuous_cols=preprocessor.continuous_cols,
    n_layers=study_saint.best_params['n_layers'],
    n_heads=study_saint.best_params['n_heads'],
    dropout=study_saint.best_params['dropout'],
)

trainer = Trainer(saint_model, objective="regression", metrics=[nn.MSELoss()])

trainer.fit(
    X_train=X_train_saint,
    target=y_train,
    n_epochs=study_saint.best_params['n_epochs'],
    #batch_size=study_saint.best_params['batch_size'],
    val_split=0.1,
)

y_test_hat_saint = trainer.predict(X_test_saint)
std_dev_error = np.std(y_test - y_test_hat_saint)
crps_saint = [crps_gaussian(y_test_np[i], mu=y_test_hat_saint[i], sig=std_dev_error) for i in range(len(y_test_hat_saint))]
crps_saint = np.mean(crps_saint)
print("CRPS SAINT: ", crps_saint)

[I 2024-01-27 19:03:48,527] A new study created in memory with name: no-name-5bc6fe89-4443-43b2-96ce-9b768f459b96
[W 2024-01-27 19:03:48,810] Trial 0 failed with parameters: {'n_layers': 4, 'n_heads': 1, 'embed_dim': 321, 'dropout': 0.7488038825386119} because of the following error: TypeError("TabTransformer.__init__() got an unexpected keyword argument 'embed_dim'").
Traceback (most recent call last):
  File "c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\dalma\AppData\Local\Temp\ipykernel_28528\648684132.py", line 25, in Saint_opt
    saint_model = TabTransformer(
TypeError: TabTransformer.__init__() got an unexpected keyword argument 'embed_dim'
[W 2024-01-27 19:03:48,810] Trial 0 failed with value None.


TypeError: TabTransformer.__init__() got an unexpected keyword argument 'embed_dim'

In [29]:
y_val_tensor.reshape(-1,1) - y_val_hat_engression

tensor([[ -3.8883],
        [ -9.3065],
        [ -2.5412],
        ...,
        [ -3.7598],
        [ -9.0308],
        [-12.7551]])

In [2]:
N_SAMPLES=100
seed=10
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
random.seed(seed)

dtrain = lgb.Dataset(torch.tensor(X_train.values, dtype=torch.float32).clone().detach(), label=y_train.values)
opt_params = study_boost.best_params.copy()
n_rounds = opt_params["n_estimators"]
del opt_params["n_estimators"]
opt_params['feature_pre_filter']=False
# Use LightGBMLossGuideRegressor for distributional prediction
boosted_tree_model = LightGBMLSS(Gaussian(stabilization="None", response_fn="exp", loss_fn="nll"))
torch.manual_seed(10)
boosted_tree_model.train(opt_params, dtrain, num_boost_round=n_rounds)
# Predict both the mean and standard deviation
pred_params=boosted_tree_model.predict(X_test, pred_type="parameters")
y_test_hat_boost=pred_params['loc']
y_test_hat_std = pred_params['scale']
# Calculate the CRPS for each prediction
crps_values = [crps_gaussian(y_test_np[i], mu=y_test_hat_boost[i], sig=y_test_hat_std[i]) for i in range(len(y_test))]
# Return the mean CRPS as the objective to be minimized
CRPS_boosted=np.mean(crps_values)

'''
rf_model.fit(X_train, y_train)
y_test_hat_rf=rf_model.predict(X_test)
# Calculate the standard deviation of the residuals
std_dev = np.std(y_test - y_test_hat_rf)
# Calculate the CRPS for each prediction
y_test_np = y_test.values.flatten()
crps_values = [crps_gaussian(y_test_np[i], mu=y_test_hat_rf[i], sig=std_dev) for i in range(len(y_test_np))]
CRPS_rf = np.mean(crps_values)'''

drf_model=drf(**study_drf.best_params)
drf_model.fit(X_train, y_train)
# Generate a sample from the drf model for each data point
y_test_hat_drf=drf_model.predict(newdata = X_test, functional = "quantile", quantiles=list(np.random.uniform(0,1,N_SAMPLES)))
# Calculate the CRPS for each prediction
crps_values = [crps_ensemble(y_test_np[i], y_test_hat_drf.quantile[i].reshape(-1)) for i in range(len(y_test_np))]
# Return the mean CRPS as the objective to be minimized
CRPS_rf=np.mean(crps_values)

lin_reg=LinearRegression()
lin_reg.fit(X_train, y_train)
y_test_hat_linreg=lin_reg.predict(X_test)
# Calculate the standard deviation of the residuals
std_dev = np.std(y_test - y_test_hat_linreg)
# Calculate the CRPS for each prediction
crps_values = [crps_gaussian(y_test_np[i], mu=y_test_hat_linreg[i], sig=std_dev) for i in range(len(y_test_np))]
CRPS_linreg = np.mean(crps_values)

params=study_engression.best_params
params['noise_dim']=params['hidden_dim']
X_train_tensor = torch.Tensor(np.array(X_train))
y_train_tensor = torch.Tensor(np.array(y_train).reshape(-1,1))
# Check if CUDA is available and if so, move the tensors and the model to the GPU
if torch.cuda.is_available():
    X_train_tensor = X_train_tensor.cuda()
    y_train_tensor = y_train_tensor.cuda()
    engressor_model=engression(X_train_tensor, y_train_tensor, lr=params['learning_rate'], num_epoches=params['num_epoches'],num_layer=params['num_layer'], hidden_dim=params['hidden_dim'], noise_dim=params['noise_dim'], batch_size=1000).cuda()
else:
    engressor_model=engression(X_train_tensor, y_train_tensor, lr=params['learning_rate'], num_epoches=params['num_epoches'],num_layer=params['num_layer'], hidden_dim=params['hidden_dim'], noise_dim=params['noise_dim'], batch_size=1000)
# Generate a sample from the engression model for each data point
y_test_hat_engression_samples = [engressor_model.sample(torch.Tensor(np.array([X_test.values[i]])).cuda() if torch.cuda.is_available() else torch.Tensor(np.array([X_test.values[i]])), sample_size=N_SAMPLES) for i in range(len(X_test))]
# Calculate the CRPS for each prediction
crps_values = [crps_ensemble(y_test_np[i], np.array(y_test_hat_engression_samples[i]).reshape(-1,)) for i in range(len(y_test_np))]
CRPS_engression=np.mean(crps_values)


print("CRPS linear regression: ",CRPS_linreg)
print("CRPS boosted trees", CRPS_boosted)
print("CRPS random forest", CRPS_rf)
print("CRPS engression", CRPS_engression)

NameError: name 'X_train' is not defined

#### Create table with results

In [43]:
crps_FTTrans=None
crps_results = {'GP': CRPS_GP, 'MLP': crps_MLP, 'ResNet': crps_ResNet, 'FTTrans': crps_FTTrans, 'boosted_trees': CRPS_boosted, 'drf': CRPS_rf, 'linear_regression': CRPS_linreg, 'engression': CRPS_engression}  # Add all your methods here

# Convert the dictionary to a DataFrame
df = pd.DataFrame(list(crps_results.items()), columns=['Method', 'CRPS'])

# Save the DataFrame to a CSV file
df.to_csv(f'RESULTS/CLUSTERING/{task_id}_clustering_crps_results.csv', index=False)

## EXTRA

#### Gaussian process with stochastic variational inference

In [ ]:
N_TRIALS=2

def SVGP_opt(trial):

    seed=10
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

    class SVGPMODEL(gpytorch.models.ApproximateGP):
        def __init__(self, inducing_points):
            variational_distribution = CholeskyVariationalDistribution(inducing_points.size(0))
            variational_strategy = VariationalStrategy(self, inducing_points, variational_distribution, learn_inducing_locations=True)
            super(SVGPMODEL, self).__init__(variational_strategy)
            self.mean_module = gpytorch.means.ConstantMean()
            self.covar_module = gpytorch.kernels.ScaleKernel(
                gpytorch.kernels.MaternKernel(nu=1.5, ard_num_dims=inducing_points.shape[1]),
                ard_num_dims=inducing_points.shape[1]
            )

        def forward(self, x):
            mean_x = self.mean_module(x)
            covar_x = self.covar_module(x)
            return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

    # Convert data to PyTorch tensors
    X_train_tensor = torch.tensor(X_train_.values, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train_.values, dtype=torch.float32)

    # Initialize the Gaussian Process model and likelihood
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model = SVGPMODEL(X_train_tensor)

    # Set the model in training mode
    model.train()
    likelihood.train()

    # Define the learning params
    n_epochs=5 #trial.suggest_int('n_epochs', 100, 5000)
    learning_rate=trial.suggest_float('learning_rate', 0.0001, 0.05, log=True)

    # Use the negative log likelihood as the loss
    mll = gpytorch.mlls.VariationalELBO(likelihood, model, num_data=y_train_tensor.numel())
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    for epoch in range(n_epochs):
        optimizer.zero_grad()
        output = model(X_train_tensor)
        loss = -mll(output, y_train_tensor)
        loss.backward()
        optimizer.step()

    # Set the model in evaluation mode
    model.eval()
    likelihood.eval()

    # Make predictions on the validation set
    with torch.no_grad(), gpytorch.settings.fast_pred_var():
        y_pred = model(torch.tensor(X_val.values, dtype=torch.float32))

    # Calculate CRPS
    y_val_np = y_val.values.flatten()
    y_pred_np = y_pred.mean.numpy().flatten()
    y_pred_std_np = y_pred.stddev.numpy().flatten()
    
    # Calculate the CRPS for each prediction
    crps_values = [crps_gaussian(y_val_np[i], mu=y_pred_np[i], sig=y_pred_std_np[i]) for i in range(len(y_val_np))]

    # Calculate the mean CRPS
    mean_crps= np.mean(crps_values)
    
    return mean_crps

sampler_SVGP = optuna.samplers.TPESampler(seed=10)
study_SVGP = optuna.create_study(sampler=sampler_SVGP, direction='minimize')
study_SVGP.optimize(SVGP_opt, n_trials=N_TRIALS)

[I 2024-01-22 17:10:06,859] A new study created in memory with name: no-name-bea4cf5f-3031-4dec-a045-a078b921cc60


[W 2024-01-22 17:18:36,997] Trial 0 failed with parameters: {'lengthscale': 0.08747537025773001, 'learning_rate': 0.00011376505702653915} because of the following error: AxisError(-1, 0, None).
Traceback (most recent call last):
  File "c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\dalma\AppData\Local\Temp\ipykernel_14344\2994460075.py", line 77, in SVGP_opt
    crps_values = [CRPS(y_val_np[i], pred_distributions[i]) for i in range(len(y_val_np))]
  File "C:\Users\dalma\AppData\Local\Temp\ipykernel_14344\2994460075.py", line 77, in <listcomp>
    crps_values = [CRPS(y_val_np[i], pred_distributions[i]) for i in range(len(y_val_np))]
  File "c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\CRPS\CRPS.py", line 109, in __init__
    self.fc = np.sort(ensemble_members)
  File "c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\numpy\cor

AxisError: axis -1 is out of bounds for array of dimension 0

In [ ]:
# Access the best parameters
best_params_SVGP = study_SVGP.best_params
lengthscale = best_params_SVGP['lengthscale']
n_epochs = best_params_SVGP['n_epochs']
learning_rate = best_params_SVGP['learning_rate']

class SVGPMODEL(gpytorch.models.ApproximateGP):
    def __init__(self, inducing_points):
        variational_distribution = CholeskyVariationalDistribution(inducing_points.size(0))
        variational_strategy = VariationalStrategy(self, inducing_points, variational_distribution, learn_inducing_locations=True)
        super(SVGPMODEL, self).__init__(variational_strategy)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.MaternKernel(nu=1.5, ard_num_dims=inducing_points.shape[1], lengthscale=lengthscale),
            ard_num_dims=inducing_points.shape[1]
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# Convert data to PyTorch tensors
X_tensor = torch.tensor(X_train, dtype=torch.float32)
y_tensor = torch.tensor(y_train, dtype=torch.float32)

# Initialize the final Gaussian Process model with the best parameters
likelihood = gpytorch.likelihoods.GaussianLikelihood()
final_model = SVGPMODEL(X_tensor)

# Set the model in training mode
final_model.train()
likelihood.train()

# Use the negative log likelihood as the loss
mll = gpytorch.mlls.VariationalELBO(likelihood, final_model, num_data=y_tensor.numel())
optimizer = torch.optim.Adam(final_model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(n_epochs):
    optimizer.zero_grad()
    output = final_model(X_tensor)
    loss = -mll(output, y_tensor)
    loss.backward()
    optimizer.step()

# Set the final model in evaluation mode
final_model.eval()
likelihood.eval()

# Make predictions on the validation set
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    y_pred = final_model(torch.tensor(X_test.values, dtype=torch.float32))


# Calculate CRPS
y_test_np = y_test.values.flatten()
y_pred_np = y_pred.mean.numpy().flatten()
y_pred_std_np = y_pred.stddev.numpy().flatten()

# Calculate the CRPS for each prediction
crps_values = [crps_gaussian(y_test_np[i], mu=y_pred_np[i], sig=y_pred_std_np[i]) for i in range(len(y_test_np))]

# Calculate the mean CRPS
CRPS_SVGP= np.mean(crps_values)

print("CRPS SVGP: ", CRPS_SVGP)

NameError: name 'study_SVGP' is not defined